# Adjacency matrix math

## Dietz et all paper

Generate a random array of areas (age x patch)

In [16]:
areas_age_patch = rand(1.0:10.0, (10,20))

10×20 Array{Float64,2}:
 8.0  10.0   4.0   3.0  10.0   7.0   6.0  …  1.0   4.0  9.0  5.0  6.0   4.0
 6.0  10.0   1.0   7.0   5.0   7.0   7.0     3.0   1.0  5.0  7.0  9.0   2.0
 7.0   9.0   3.0   4.0   2.0   8.0   2.0     9.0   1.0  9.0  7.0  5.0  10.0
 3.0   7.0   6.0   1.0   3.0  10.0  10.0     3.0   5.0  3.0  2.0  5.0  10.0
 9.0   2.0   9.0   1.0   4.0   2.0   5.0     2.0   5.0  4.0  9.0  3.0   6.0
 7.0   1.0   2.0   5.0   2.0   1.0   3.0  …  2.0   7.0  5.0  9.0  2.0   9.0
 3.0  10.0   8.0   4.0   8.0   5.0   1.0     8.0   4.0  1.0  8.0  5.0   8.0
 8.0   9.0  10.0   2.0   1.0   8.0  10.0     1.0  10.0  5.0  8.0  7.0   5.0
 4.0   4.0   3.0   4.0   8.0   7.0   6.0     7.0   6.0  7.0  3.0  6.0   6.0
 7.0   6.0   4.0  10.0   2.0   6.0   5.0     1.0   3.0  1.0  1.0  2.0   6.0

Calculate the _fraction of areas for each age class_

In [20]:
fracarea_age = sum.(eachrow(areas_age_patch./sum(areas_age_patch)))

10-element Array{Float64,1}:
 0.113345521023766
 0.10603290676416818
 0.09597806215722121
 0.10237659963436929
 0.08226691042047529
 0.08775137111517367
 0.10420475319926875
 0.11151736745886655
 0.10877513711151736
 0.08775137111517368

Generate a random vector of _probability of disturbance initiation_

In [22]:
p_0 = rand(10)

10-element Array{Float64,1}:
 0.7132765570312911
 0.7459773564231225
 0.23896500723265746
 0.3895479077461912
 0.19361586160013067
 0.9212398488967886
 0.6972295698746003
 0.5570830175945638
 0.4930659191993949
 0.4811556774105499

Calculate the _initial disturbance area_.  I think this is actually initial disturbance area _fraction_.

In [23]:
I_1 = p_0.*fracarea_age

10-element Array{Float64,1}:
 0.08084670299074963
 0.0790981474817936
 0.022935398317576815
 0.03988059018973804
 0.015928178742241093
 0.08084005986662861
 0.07265463525201503
 0.062124431578187186
 0.05363331296593052
 0.042222070412625955

Generate an **initial adjacency matrix `A_0`**.  Right now this is random; make this a geometric decay distribution later.

In [56]:
A_0 = rand(Float64,(10,10));

Set a scalar parameter for a simple **probability of spread across age class `p_s`**

In [24]:
p_s = 0.1

0.1

Calculate **probability of distribution spread to h patches, `I`**

In [71]:
I = Array{Float64}(undef,(10,10))
for h = 1:10
    I[h,:] = ((p_s*A_0).^h)*I_1
end

Calculate **overall disturbance rate, `D`**

In [72]:
D = sum.(eachrow(I))

10-element Array{Float64,1}:
 0.26351614045409855
 0.017004249553559846
 0.0012640568532815996
 0.00010140352509934662
 8.511737616967833e-6
 7.360454371828044e-7
 6.500972629421475e-8
 5.833993083449059e-9
 5.30108604245238e-10
 4.865416792630755e-11

Calculate **size distribution, `p(h)`**

In [ ]:
function p(h,I)
    
end

## Initial explorations

Generate a random array of patches areas.  We'll say that the dimensions are patch age x patch 'number'.  There may not be an equivalent number of patches for each age bin (row) in reality.  The maximum number of age rows would be defined by `nlevage`.

In [1]:
areas = rand(1.0:10.0, (3,5))

3×5 Array{Float64,2}:
 5.0  9.0  3.0  4.0  4.0
 2.0  9.0  5.0  9.0  9.0
 8.0  7.0  7.0  4.0  2.0

Calculate the total area in each given patch age bin.  This is equivalent to `ed_site_type%area_by_age` (which doesn't appear to have a direct history variable output).

In [5]:
area_age = sum(areas, dims=2)
# area_age1 = sum.(eachrow(areas))

3×1 Array{Float64,2}:
 25.0
 34.0
 28.0

Calulcate the total patch area for the given site.  This is equivalent to the history variable `FATES_PATCHAREA_AP`.

In [6]:
area_tot = sum(areas)

87.0

Calculate the patch area fraction within the age bin its associated with.  Confirm that the values sum to unity.

In [7]:
# I'm pretty sure Julia has a way to easily broadcast this in one line instead of using a for loop.  Or maybe use a map?
agefrac = Array{Float64}(undef,size(areas))
for i = 1:size(areas)[1]
    agefrac[i,:] = (areas[i,:].*inv(area_age[i]))'
    println(sum(agefrac[i,:]))
end
# inv.(area_age).*collect(eachcol(areas))' # eachcol will create iterator over columns

1.0
1.0
1.0


In [8]:
agefrac

3×5 Array{Float64,2}:
 0.2        0.36      0.12      0.16      0.16
 0.0588235  0.264706  0.147059  0.264706  0.264706
 0.285714   0.25      0.25      0.142857  0.0714286

***How do we use the agefrac to determine a value for self-similar adjacency?***

In [9]:
# Individual fraction of the total area for a given patch
totfrac = areas.*inv.(area_tot)

3×5 Array{Float64,2}:
 0.0574713  0.103448   0.0344828  0.045977  0.045977
 0.0229885  0.103448   0.0574713  0.103448  0.103448
 0.091954   0.0804598  0.0804598  0.045977  0.0229885

***How do we use the totfrac to determine a value for non-similar adjacency?***

In [10]:
# The fraction of the total area for a given patch age
sum(totfrac,dims=2)

3×1 Array{Float64,2}:
 0.28735632183908044
 0.3908045977011494
 0.32183908045977017

In [11]:
# The fraction of the total area for a given patch age
sum(sum(totfrac,dims=2))

1.0

Since the size of patches varies, we will need to determine variance in the sizes

In [13]:
findmin(areas)

(2.0, CartesianIndex(2, 1))

In [14]:
findmax(areas)

(9.0, CartesianIndex(1, 2))

In [18]:
using Statistics

In [19]:
mean(areas[1,:])

5.0

In [23]:
areas

3×5 Array{Float64,2}:
 5.0  9.0  3.0  4.0  4.0
 2.0  9.0  5.0  9.0  9.0
 8.0  7.0  7.0  4.0  2.0